#### **Step - 1**

##### Set Data Ingestion Configuration in config folder for config.yaml file

#### **Step - 2**

##### Set Data Ingestion Schema in schema.yaml file

#### **Step - 3**

##### Set Data Ingestion params in parms.yaml file

#### **Step - 4**

##### Set Data Ingestion Entity which is just return type.

In [2]:
import os

In [3]:
%pwd

'/Users/sudhirmalik/Documents/End_to_End_Projects/Jamboree_MLOPS/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Users/sudhirmalik/Documents/End_to_End_Projects/Jamboree_MLOPS'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from src.JambooreProject.constants import *
from src.JambooreProject.utils.common import create_directories, read_yaml 

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import urllib.request as request
import zipfile
from src.JambooreProject import logger 
from src.JambooreProject.utils.common import * 

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-23 23:34:48,201]: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-23 23:34:48,207]: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-23 23:34:48,209]: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-23 23:34:48,211]: INFO: common: created directory at: artifacts]
[2024-01-23 23:34:48,213]: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-23 23:34:49,674]: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 5929
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "7e201fe20d3640eb8f1f149dd22ec18a47c4ce8e4a718ad2f81027077cbfa613"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4EE0:78D4:1DCC0:23D9C:65AFFFBF
Accept-Ranges: bytes
Date: Tue, 2